In [1]:
import cv2
import os
import timm
import numpy as np
import pandas as pd
import albumentations as A
import logging

from glob import glob
from tqdm import tqdm
from easydict import EasyDict
from albumentations.pytorch import ToTensorV2
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import f1_score

import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import CosineAnnealingLR

# custom dataset

In [2]:
class CustomDataset(Dataset):
    def __init__(self, img_list, label_list=None, transforms=None, mode="train") :
        self.img_list = img_list
        
        if mode == "train" : 
            self.label_list = self.label_encoder(label_list)
            
        self.transforms = transforms
        self.mode = mode
    def __len__(self):
        return len(self.img_list)
    
    def __getitem__(self, idx):
        img_path = self.img_list[idx]
        
#         img = cv2.imread("../data/train/"+img_path)
        img = cv2.imread(img_path)
        
        try :
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        except :
            print(img_path)
            img = cv2.imread(img_path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            print(img)
            print(img.shape)
            
        if self.transforms:            
            img = self.transforms(image=img)['image']
        
        if self.mode == "train" :
            label = self.label_list[idx]
            return img, torch.tensor(label)
        
        elif self.mode == "test" :
            return img
    
    def label_encoder(self, label_list) :
        label_enc = {k : i for i, k in enumerate(sorted(list(set(label_list))))}
#         display(label_enc)
        return [label_enc[label] for label in label_list]

#### test code

In [3]:
# df = pd.read_csv('../data/aug_train_df.csv')
# transforms = A.Compose([
#     A.Resize(224,224),
#     A.Normalize(),
#     A.Rotate(),
#     ToTensorV2()
# ])
# db = CustomDataset(list(df['file_name']), list(df['label']), transforms, mode="train")
# db_loader = DataLoader(db, batch_size=16, shuffle=True)
# for img, label in db_loader : 
#     print(img.shape)
#     print(label.shape)
#     print(label)
#     break

# Focal Loss

In [4]:
# class FocalLoss(nn.Module) :
#     def __init__(self, alpha=2, gamma=2, logits=False, reduction='none') :
#         super(FocalLoss, self).__init__()
#         self.alpha = alpha
#         self.gamma = gamma
#         self.logits = logits
#         self.reduction = reduction

#     def forward(self, inputs, targets) :
#         ce_loss = nn.CrossEntropyLoss(reduction=self.reduction)(inputs, targets)
#         pt = torch.exp(-ce_loss)
#         F_loss = self.alpha * (1-pt)**self.gamma * ce_loss

#         if self.reduction :
#             return torch.mean(F_loss)
#         else :
#             return F_loss

# model

In [5]:
class CNN(nn.Module) :
    def __init__(self, model_name, num_classes) :
        super(CNN, self).__init__()
        self.model = timm.create_model(model_name=model_name, num_classes=num_classes, pretrained=True)
    
    def forward(self, x) :
        output = self.model(x)
        return output

# Custom SwinTransformer

In [6]:
class BackBone(nn.Module) :
    def __init__(self, model_name, backbone_output) :
        super(BackBone, self).__init__()
        self.model = timm.create_model(model_name=model_name, num_classes=backbone_output, pretrained=True)
    
    def forward(self, x) :
        output = self.model(x)
        return output
    
class MLP(nn.Module) :
    def __init__(self, in_features, dropout_rate, num_state) :
        super(MLP, self).__init__()
        #forward_features 시 LayerNorm까지 통과한 결과임
        # 따라서 LayerNorm 와 AdaptiveAvgPool1d는 필요없음    

        self.linear_1 = nn.Linear(in_features, in_features//2, bias=True)
        self.gelu = nn.GELU()
        self.dropout = nn.Dropout(p=dropout_rate, inplace=False)
        self.linear_2 = nn.Linear(in_features//2, num_state, bias=True)
        
    def forward(self, x):
        x = self.linear_1(x)
        x = self.gelu(x)
        x = self.dropout(x)
        x = self.linear_2(x)
        return x
    
class CustomSwinTransformer(nn.Module) :
    def __init__(self, 
                 model_path, 
                 model_name, 
                 backbone_output, 
                 num_class, 
                 num_state,
                 dropout_rate=0.5) :
        super(CustomSwinTransformer, self).__init__()
        
        self.backbone = self.get_backbone(model_path,
                                         model_name,
                                         backbone_output)
        
        # num_state + 1을 해준 이유 = None Class를 추가할 예정이기 때문
        self.mlps = nn.ModuleList([MLP(in_features=1024, 
                         dropout_rate=dropout_rate, 
                         num_state = num_state[i] + 1) for i in range(num_class)])
        
    def forward(self, x) :
        preds = []
        feature_map = self.backbone.forward_features(x)
        for mlp in self.mlps :
            preds.append(mlp(feature_map))
        return preds
    
    def get_backbone(self, model_path, model_name, backbone_output) :
        checkpoint = torch.load(model_path)
        backbone = BackBone(model_name, backbone_output)
        backbone.load_state_dict(checkpoint["model_state_dict"])
        return backbone.model

#### test code

In [7]:
# model = CNN("efficientnetv2_s", 88)
# test_data = torch.randn(1, 3, 666,666)
# output = model(test_data)
# output = F.softmax(output)
# output

# Training

In [8]:
def rand_bbox(size, lam):
    W = size[2]
    H = size[3]
    cut_rat = np.sqrt(1. - lam)
    cut_w = np.int(W * cut_rat)
    cut_h = np.int(H * cut_rat)

    # uniform
    cx = np.random.randint(W)
    cy = np.random.randint(H)

    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)

    return bbx1, bby1, bbx2, bby2

def cutmix(imgs, labels):
    lam = np.random.beta(1.0, 1.0)
    rand_index = torch.randperm(imgs.size()[0]).cuda()
    target_a = labels
    target_b = labels[rand_index]
    bbx1, bby1, bbx2, bby2 = rand_bbox(imgs.size(), lam)
    imgs[:, :, bbx1:bbx2, bby1:bby2] = imgs[rand_index, :, bbx1:bbx2, bby1:bby2]

    # adjust lambda to exactly match pixel ratio
    lam = 1 - ((bbx2 - bbx1) * (bby2 - bby1) / (imgs.size()[-1] * imgs.size()[-2]))

    return imgs, target_a, target_b, lam

def mixup(imgs, labels, alpha=1.0) :
    lam = np.random.beta(alpha, alpha)
    rand_index = torch.randperm(imgs.size()[0]).cuda()
    mixed_imgs = lam * imgs + (1 - lam) * imgs[rand_index, :]
    target_a, target_b = labels, labels[rand_index]
    
    return mixed_imgs, lam, target_a, target_b

def accuracy_function(real, pred):    
    real = real.cpu()
    pred = torch.argmax(pred, dim=1).cpu()
    score = f1_score(real, pred, average='macro')
    return score

def training(model, train_loader, valid_loader, opt) :
        
    optimizer = torch.optim.AdamW(model.parameters(), lr= opt.learning_rate,
                                 weight_decay=opt.weight_decay)
    scheduler = CosineAnnealingLR(optimizer, 
                                  T_max=opt.cosine_lr_Tmax, 
                                  eta_min=opt.cosine_lr_eta_min)
    
    criterion = nn.CrossEntropyLoss()
#     criterion = FocalLoss()
    
    if opt.resume : 
        model_checkpoint = torch.load(opt.model_path)
        model.load_state_dict(model_checkpoint["model_state_dict"])
        optimizer.load_state_dict(model_checkpoint["optimizer_state_dict"])
        scheduler = CosineAnnealingLR(optimizer, 
                                  T_max=opt.cosine_lr_Tmax, 
                                  eta_min=opt.cosine_lr_eta_min)
        
        opt.start_epoch = model_checkpoint["epoch"]
    else : 
        opt.start_epoch = 0
        
    early_stop_step = 0
    best_loss = 10
    for E in range(opt.start_epoch + 1, opt.epochs + 1) :
            # train
            tqdm_train = tqdm(train_loader)
            train_loss, train_f1 = 0, 0
            for batch, (img, label) in enumerate(tqdm_train, start=1) :
                model.train()
                optimizer.zero_grad()
                
                img = img.to(opt.device)
                label = label.to(opt.device)
                
                if opt.cutmix :
                    imgs, target_a, target_b, lam = cutmix(img, label)
                    output = model(imgs)
                    loss = criterion(output, target_a) * lam + criterion(output, target_b) * (1. - lam)
                
                elif opt.mixup :
                    mixed_imgs, lam, target_a, target_b = mixup(img, label)
                    output = model(mixed_imgs)
                    loss = criterion(output, target_a) * lam + criterion(output, target_b) * (1. - lam)                    
                
                else:
                    output = model(img)
                    loss = criterion(output, label)
                
                loss.backward()
                optimizer.step()
                
                score = accuracy_function(label, output)
                train_loss += loss.item()
                train_f1 += score
                tqdm_train.set_postfix({"Epoch" : E,
                                "Mean train loss" : "{:06f}".format(train_loss / (batch)),
                                "Mean train f1" : "{:06f}".format(train_f1 / (batch))
                               })
            # validation
            tqdm_valid = tqdm(valid_loader)
            valid_loss, valid_f1 = 0, 0
            for batch, (img, label) in enumerate(tqdm_valid, start=1) :
                model.eval()
                
                img = img.to(opt.device)
                label = label.to(opt.device)
                
                with torch.no_grad() :
                    output = model(img)
                    loss = criterion(output, label)
                    
                score = accuracy_function(label, output)
                valid_loss += loss.item()
                valid_f1 += score
                tqdm_valid.set_postfix({
                    "Mean valid loss": "{:06f}".format(valid_loss / (batch)),
                    "Mean valid f1": "{:06f}".format(valid_f1 / (batch))
                    })
            
            # scheduler
            scheduler.step()
            
            mean_valid_loss = valid_loss / batch
            if mean_valid_loss < best_loss :
                early_stop_step = 0
                best_loss = mean_valid_loss
                if opt.kfold_step is not None :
                    torch.save({
                        "epoch" : E,
                        "model_state_dict" : model.state_dict(),
                        "optimizer_state_dict" : optimizer.state_dict()
                    }, 
                               os.path.join(opt.save_path, f'{E}E_{mean_valid_loss:0.4f}_{opt.kfold_step}fold_{opt.model_name}.pt'))
                else :
                    torch.save({
                        "epoch" : E,
                        "model_state_dict" : model.state_dict(),
                        "optimizer_state_dict" : optimizer.state_dict()
                    }, 
                               os.path.join(opt.save_path, f'{E}E_{mean_valid_loss:0.4f}_{opt.model_name}.pt'))
            
            elif mean_valid_loss > best_loss : 
                early_stop_step += 1
                print(f"Early Stopping Step : [{early_stop_step} / {opt.early_stopping}]")
            
            if early_stop_step == opt.early_stopping :
                print("=== Early Stop ===")
                break

In [9]:
def create_log(save_path, name=None) :
    logger = logging.getLogger(name)
    logger.setLevel(logging.DEBUG)
    
    file_handler = logging.FileHandler(filename=os.path.join(save_path, "train.log"))

    # formatter 객체 생성
    formatter = logging.Formatter(fmt="%(asctime)s - %(message)s", 
                                  datefmt='%Y-%d-%m %H:%M:%S')

    # handler에 level 설정
    file_handler.setLevel(logging.DEBUG)

    # handler에 format 설정
    file_handler.setFormatter(formatter)
    
    # logger에 handler 추가
    logger.addHandler(file_handler)
    
    return logger

# Run - efficientNetV2_s

In [10]:
opt = {
    "df_path" : "../data/aug_v4_train_df.csv",
    "save_path" : "../model/effiv2s_aug_v4_mixup",
    "model_name" : "efficientnetv2_rw_s",
    "num_classes" : 88,
    "resize" : 224,
    "device" : "cuda:0",
    "early_stopping" : 5,
    "epochs" : 30,
    "batch_size" : 32,
    "learning_rate" : 1e-4,
    "weight_decay" : 0.01,
    "cosine_lr_Tmax" : 20,
    "cosine_lr_eta_min" : 1e-5,
    "cutmix" : False,
    "mixup" : False,
    "resume" : False,
    "model_path" : "../model/effiv2s_aug_v4_mixup/1E_0.8879_1fold_efficientnetv2_rw_s.pt",
    "logging" : True,
    "kfold" : 4
}
opt = EasyDict(opt)
os.makedirs(opt.save_path, exist_ok=True)

#  option logging
log = create_log(opt.save_path)
log.info(opt)

t_transforms = A.Compose([
    A.Normalize(),
    A.Resize(opt.resize, opt.resize),
    A.Blur(blur_limit=(10, 10), p=0.7),
    A.Rotate(limit=(-75, 75), p=1),
#     A.OneOf([
#         A.HorizontalFlip(),
#         A.VerticalFlip()
#     ], p=1),
    ToTensorV2()
])

v_transforms = A.Compose([
    A.Normalize(),
    A.Resize(opt.resize, opt.resize),
    ToTensorV2()
])

train_df = pd.read_csv(opt.df_path)
if opt.kfold is None :
    t_imgs, v_imgs, t_labels, v_labels = train_test_split(
    list(train_df['file_name']),
    list(train_df['label']),
    train_size=0.8,
    shuffle=True,
    random_state=51,
    stratify=list(train_df['label']))
    
    train_data = CustomDataset(t_imgs, t_labels, t_transforms)
    valid_data = CustomDataset(v_imgs, v_labels, v_transforms)

    train_loader = DataLoader(train_data, batch_size=opt.batch_size, shuffle=True)
    valid_loader = DataLoader(valid_data, batch_size=opt.batch_size, shuffle=True)


    # custom_swin = CustomSwinTransformer(**model_opt).to(opt.device)
    # model = WeightFreeze(custom_swin)
    model = CNN(opt.model_name, opt.num_classes).to(opt.device)

    training(model, train_loader, valid_loader, opt)

    print("==== Complete ====")

# K - fold Training
else :
    imgs_list = list(train_df['file_name'])
    labels_list = list(train_df['label'])

    kfold = StratifiedKFold(n_splits=opt.kfold, random_state=13, shuffle=True)
    for k, (fold_train, fold_valid) in enumerate(kfold.split(imgs_list, labels_list), 1) :
        
        t_imgs = list(map(lambda x : imgs_list[x] , fold_train))
        t_labels = list(map(lambda x : labels_list[x] , fold_train))

        v_imgs = list(map(lambda x : imgs_list[x] , fold_valid))
        v_labels = list(map(lambda x : labels_list[x] , fold_valid))
        
        print(f"\n\n\n===== k_fold : {k} / {opt.kfold} =====")       
        if k == 1 or k == 2:
            continue

        train_data = CustomDataset(t_imgs, t_labels, t_transforms)
        valid_data = CustomDataset(v_imgs, v_labels, v_transforms)

        train_loader = DataLoader(train_data, batch_size=opt.batch_size, shuffle=True)
        valid_loader = DataLoader(valid_data, batch_size=opt.batch_size, shuffle=True)

        model = CNN(opt.model_name, opt.num_classes).to(opt.device)

#         opt.model_name = str(k)+"fold_"+opt.model_name
        opt.kfold_step = k

        training(model, train_loader, valid_loader, opt)
        
    print("==== Complete ====")




===== k_fold : 1 / 4 =====



===== k_fold : 2 / 4 =====



===== k_fold : 3 / 4 =====


 32%|██████▋              | 231/726 [04:00<08:33,  1.04s/it, Epoch=6, Mean train loss=0.122263, Mean train f1=0.928777]

../data/train\aug_v4\capsule-faulty_imprint\aug_2_12094.png
[[[220 217 217]
  [217 215 216]
  [212 215 214]
  ...
  [222 213 217]
  [225 212 217]
  [228 213 215]]

 [[218 217 218]
  [217 217 217]
  [214 215 216]
  ...
  [220 213 220]
  [224 212 221]
  [228 213 219]]

 [[215 216 219]
  [217 217 217]
  [215 216 215]
  ...
  [218 212 216]
  [219 213 219]
  [220 215 220]]

 ...

 [[159 163 155]
  [157 162 154]
  [159 161 154]
  ...
  [203 206 199]
  [204 207 202]
  [205 208 204]]

 [[156 161 153]
  [155 162 152]
  [156 162 153]
  ...
  [206 206 201]
  [207 209 205]
  [207 211 206]]

 [[155 160 152]
  [155 163 151]
  [157 162 152]
  ...
  [207 207 200]
  [207 209 202]
  [208 210 204]]]
(1000, 1000, 3)


100%|██████████████████████████████| 242/242 [03:29<00:00,  1.16it/s, Mean valid loss=0.219407, Mean valid f1=0.896446]


Early Stopping Step : [1 / 5]


...

../data/train\10624.png
[[[ 72  56  62]
  [ 76  69  80]
  [ 78  80  79]
  ...
  [ 89  92  92]
  [ 80  82  88]
  [ 71  77  81]]

 [[ 57  59  67]
  [ 73  73  74]
  [ 80  78  81]
  ...
  [102  88  88]
  [ 91  84  84]
  [ 82  81  83]]

 [[ 63  68  70]
  [ 70  68  70]
  [ 75  71  78]
  ...
  [ 94  85  86]
  [ 94  84  84]
  [ 89  84  84]]

 ...

 [[ 79  74  78]
  [ 83  77  81]
  [ 85  79  78]
  ...
  [ 30  32  36]
  [ 33  32  37]
  [ 31  34  40]]

 [[ 77  73  76]
  [ 83  77  75]
  [ 87  83  83]
  ...
  [ 27  26  28]
  [ 32  28  29]
  [ 27  29  32]]

 [[ 70  65  73]
  [ 75  72  75]
  [ 78  78  84]
  ...
  [ 30  28  33]
  [ 40  31  34]
  [ 31  30  39]]]
(1024, 1024, 3)


100%|██████████████████████████████| 242/242 [03:31<00:00,  1.14it/s, Mean valid loss=0.080826, Mean valid f1=0.976838]


Early Stopping Step : [1 / 5]


  2%|▍                    | 16/726 [00:17<13:10,  1.11s/it, Epoch=21, Mean train loss=0.006984, Mean train f1=0.997024]


KeyboardInterrupt: 

# Run - coat_mini

In [ ]:
# opt = {
#     "df_path" : "../data/aug_train_df.csv",
#     "save_path" : "../model/coat_mini",
#     "model_name" : "coat_mini",
#     "num_classes" : 88,
#     "resize" : 224,
#     "device" : "cuda:0",
#     "early_stopping" : 5,
#     "epochs" : 25,
#     "batch_size" : 32,
#     "learning_rate" : 1e-4,
#     "cosine_lr_Tmax" : 20,
#     "cosine_lr_eta_min" : 1e-5,
#     "cutmix" : False,
#     "resume" : True,
#     "model_path" : "../model/coat_mini/9E_0.0738_coat_mini.pt"
# }
# opt = EasyDict(opt)
# os.makedirs(opt.save_path, exist_ok=True)

# t_transforms = A.Compose([
#     A.Normalize(),
#     A.Resize(opt.resize, opt.resize),
#     A.Blur(p=0.7),
#     A.Rotate(limit=(-270, 270), p=1),
#     A.OneOf([
#         A.HorizontalFlip(),
#         A.VerticalFlip()
#     ], p=1),
#     ToTensorV2()
# ])

# v_transforms = A.Compose([
#     A.Normalize(),
#     A.Resize(opt.resize, opt.resize),
#     ToTensorV2()
# ])

# train_df = pd.read_csv(opt.df_path)
# t_imgs, v_imgs, t_labels, v_labels = train_test_split(
#     list(train_df['file_name']),
#     list(train_df['label']),
#     train_size=0.8,
#     shuffle=True,
#     random_state=51,
#     stratify=list(train_df['label']))


# train_data = CustomDataset(t_imgs, t_labels, t_transforms)
# valid_data = CustomDataset(v_imgs, v_labels, v_transforms)

# train_loader = DataLoader(train_data, batch_size=opt.batch_size, shuffle=True)
# valid_loader = DataLoader(valid_data, batch_size=opt.batch_size, shuffle=True)

# model = CNN(opt.model_name, opt.num_classes).to(opt.device)
    
# training(model, train_loader, valid_loader, opt)

In [ ]:
# import gc

# model.cpu()
# del model
# del train_data
# del valid_data
# del train_loader
# del valid_loader

# gc.collect()
# torch.cuda.empty_cache()

# Run - coat_mini w/ aug_v2

In [ ]:
# opt = {
#     "df_path" : "../data/aug_v2_train_df.csv",
#     "save_path" : "../model/coat_mini_aug_v2_CEL",
#     "model_name" : "coat_mini",
#     "num_classes" : 88,
#     "resize" : 224,
#     "device" : "cuda:0",
#     "early_stopping" : 5,
#     "epochs" : 15,
#     "batch_size" : 32,
#     "learning_rate" : 1e-4,
#     "cosine_lr_Tmax" : 20,
#     "cosine_lr_eta_min" : 1e-5,
#     "cutmix" : False,
#     "resume" : True,
#     "model_path" : "../model/coat_mini_aug_v2_CEL/5E_0.0848_coat_mini.pt"
# }
# opt = EasyDict(opt)
# os.makedirs(opt.save_path, exist_ok=True)

# t_transforms = A.Compose([
#     A.Normalize(),
#     A.Resize(opt.resize, opt.resize),
#     A.Blur(p=0.7),
#     A.Rotate(limit=(-270, 270), p=1),
#     A.OneOf([
#         A.HorizontalFlip(),
#         A.VerticalFlip()
#     ], p=1),
#     ToTensorV2()
# ])

# v_transforms = A.Compose([
#     A.Normalize(),
#     A.Resize(opt.resize, opt.resize),
#     ToTensorV2()
# ])

# train_df = pd.read_csv(opt.df_path)
# t_imgs, v_imgs, t_labels, v_labels = train_test_split(
#     list(train_df['file_name']),
#     list(train_df['label']),
#     train_size=0.8,
#     shuffle=True,
#     random_state=51,
#     stratify=list(train_df['label']))


# train_data = CustomDataset(t_imgs, t_labels, t_transforms)
# valid_data = CustomDataset(v_imgs, v_labels, v_transforms)

# train_loader = DataLoader(train_data, batch_size=opt.batch_size, shuffle=True)
# valid_loader = DataLoader(valid_data, batch_size=opt.batch_size, shuffle=True)

# model = CNN(opt.model_name, opt.num_classes).to(opt.device)
    
# training(model, train_loader, valid_loader, opt)

# print("==== Complete ====")

# Run - coat_mini w/ aug_v3

In [ ]:
# opt = {
#     "df_path" : "../data/aug_v3_train_df.csv",
#     "save_path" : "../model/coat_mini_aug_v3_CEL",
#     "model_name" : "coat_mini",
#     "num_classes" : 88,
#     "resize" : 224,
#     "device" : "cuda:0",
#     "early_stopping" : 5,
#     "epochs" : 20,
#     "batch_size" : 32,
#     "learning_rate" : 1e-4,
#     "cosine_lr_Tmax" : 20,
#     "cosine_lr_eta_min" : 1e-5,
#     "cutmix" : False,
#     "resume" : False,
#     "model_path" : "../model/coat_mini_aug_v2_CEL/5E_0.0848_coat_mini.pt"
# }
# opt = EasyDict(opt)
# os.makedirs(opt.save_path, exist_ok=True)

# t_transforms = A.Compose([
#     A.Normalize(),
#     A.Resize(opt.resize, opt.resize),
# #     A.Blur(p=0.7),
#     A.Rotate(limit=(45), p=1),
#     A.OneOf([
#         A.HorizontalFlip(),
#         A.VerticalFlip()
#     ], p=1),
#     ToTensorV2()
# ])

# v_transforms = A.Compose([
#     A.Normalize(),
#     A.Resize(opt.resize, opt.resize),
#     ToTensorV2()
# ])

# train_df = pd.read_csv(opt.df_path)
# t_imgs, v_imgs, t_labels, v_labels = train_test_split(
#     list(train_df['file_name']),
#     list(train_df['label']),
#     train_size=0.8,
#     shuffle=True,
#     random_state=51,
#     stratify=list(train_df['label']))


# train_data = CustomDataset(t_imgs, t_labels, t_transforms)
# valid_data = CustomDataset(v_imgs, v_labels, v_transforms)

# train_loader = DataLoader(train_data, batch_size=opt.batch_size, shuffle=True)
# valid_loader = DataLoader(valid_data, batch_size=opt.batch_size, shuffle=True)

# model = CNN(opt.model_name, opt.num_classes).to(opt.device)
    
# training(model, train_loader, valid_loader, opt)

# print("==== Complete ====")

In [ ]:
# import gc

# model.cpu()
# del model
# del train_data
# del valid_data
# del train_loader
# del valid_loader

# gc.collect()
# torch.cuda.empty_cache()

# Run - SwinTransformer w/ aug_v5

In [ ]:
# opt = {
#     "df_path" : "../data/aug_v5_train_df.csv",
#     "save_path" : "../model/swin_aug_v5_mixup",
#     "model_name" : "swin_base_patch4_window7_224_in22k",
#     "num_classes" : 88,
#     "resize" : 224,
#     "device" : "cuda:0",
#     "early_stopping" : 5,
#     "epochs" : 30,
#     "batch_size" : 32,
#     "learning_rate" : 1e-4,
#     "weight_decay" : 0.01,
#     "cosine_lr_Tmax" : 20,
#     "cosine_lr_eta_min" : 1e-5,
#     "cutmix" : False,
#     "mixup" : True,
#     "resume" : False,
#     "model_path" : "../model/swin_aug_v4_mixup/21E_0.0382_swin_base_patch4_window7_224_in22k.pt"
# }
# opt = EasyDict(opt)
# os.makedirs(opt.save_path, exist_ok=True)

# t_transforms = A.Compose([
#     A.Normalize(),
#     A.Resize(opt.resize, opt.resize),
#     A.Blur(p=0.7),#blur_limit=(7, 7), p=0.7),
#     A.Rotate(limit=(45), p=1),
#     A.OneOf([
#         A.HorizontalFlip(),
#         A.VerticalFlip()
#     ], p=1),
#     ToTensorV2()
# ])

# v_transforms = A.Compose([
#     A.Normalize(),
#     A.Resize(opt.resize, opt.resize),
#     ToTensorV2()
# ])

# train_df = pd.read_csv(opt.df_path)
# t_imgs, v_imgs, t_labels, v_labels = train_test_split(
#     list(train_df['file_name']),
#     list(train_df['label']),
#     train_size=0.8,
#     shuffle=True,
#     random_state=51,
#     stratify=list(train_df['label']))


# train_data = CustomDataset(t_imgs, t_labels, t_transforms)
# valid_data = CustomDataset(v_imgs, v_labels, v_transforms)

# train_loader = DataLoader(train_data, batch_size=opt.batch_size, shuffle=True)
# valid_loader = DataLoader(valid_data, batch_size=opt.batch_size, shuffle=True)

# model = CNN(opt.model_name, opt.num_classes).to(opt.device)
    
# training(model, train_loader, valid_loader, opt)

# print("==== Complete ====")